In [ ]:
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import os

def split_filepath(filepath):
    dirpath, filename = os.path.split(filepath)
    name, ext = os.path.splitext(filename)
    separator = '/' if dirpath[-1] != '/' else ''
    return dirpath + separator, name, ext

datapath_raw = "../../data/raw/zenodo"
def enumerate_dataset(folder = '/'):
    print("Enumerating folder: ", folder)
    for filename in os.listdir(datapath_raw + folder):
        filepath = folder + filename
        dirpath, name, ext = split_filepath(filepath)
        yield filepath, dirpath, name, ext
        if not ext:
            for fullpath, dirpath, name, ext in enumerate_dataset(filepath + "/"):
                yield fullpath, dirpath, name, ext


In [ ]:

img_types = [".bmp", ".png", ".tif", ".tiff"]
misc_types = [".md", ".zip", ".txt", ".csv", ".py"]

LABELED = 'Labeled'
MASK = 'Mask'
UNLABELED = 'Unlabeled'
SYNTHETIC = 'Synthetic'

file_types = {cat:{type:set() for type in (img_types + misc_types)} for cat in [LABELED, MASK, UNLABELED, SYNTHETIC]}

def categorize(dirpath):
    if any(flag in dirpath for flag in ["osilab"]):
        return 'Synthetic'
    elif any(flag in dirpath for flag in ["Hidden"]):
        return 'Unlabeled'
    elif any(flag in dirpath for flag in ["labels"]):
        return 'Mask'
    elif any(flag in dirpath for flag in ["images", "WSI"]):
        return 'Labeled'
    else:
        return 'Unlabeled'

folders = set()
zipped_folders = set()

for datapath, dirpath, name, ext in enumerate_dataset():
    if not ext:
        folders.add(datapath)
    else:
        if ext == ".zip":
            zipped_folders.add(dirpath + name)
        if ext in misc_types:
            print("Misc file found: ", datapath)
        file_types[categorize(dirpath)][ext].add(datapath)

print(zipped_folders)
for folder in zipped_folders:
    if folder not in folders:
        print("! WARNING ! Unzip: ", folder)

In [ ]:
for cat in [LABELED, MASK, UNLABELED, SYNTHETIC]:
    types = file_types[cat]
    counts = {k:len(s) for k, s in types.items()}
    print(cat, sum(counts.values()), counts)

x = np.arange(len(img_types))  # the label locations
width = 0.25

# Display parallel histograms
plt.figure()
ax = plt.subplot(2, 1, 1)

index = 0
for cat, data in file_types.items():
    plt.bar(x + index * width, [len(data[key]) for key in img_types], width, label=cat)
    index += 1
    
plt.title("File types in dataset")
plt.xlabel("File type")
plt.xticks(x + width, img_types)
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
mask_assoc = {}
visited = set(file_types[MASK][".tiff"])

def get_mask_path(img_path):
    folder, name, ext = split_filepath(img_path)
    if "WSI" in folder:
        return folder.replace("/WSI", "/WSI-labels") + name + "_label.tiff"
    if "/images" in folder:
        return folder.replace("/images", "/labels") + name + "_label.tiff"
    return None

for ext in img_types:
    for path in file_types[LABELED][ext]:
        mask_path = get_mask_path(path)
        if mask_path and os.path.exists(datapath_raw + mask_path):
            mask_assoc[path] = mask_path
            visited.remove(mask_path)
        else:
            print("Missing mask: ", datapath_raw, mask_path, path)

print(len(mask_assoc))
print(visited)
assert not visited

In [ ]:
synth_assoc = {}
visited = set(file_types[SYNTHETIC][".tiff"])

def get_synth_path(img_path):
    folder, name, ext = split_filepath(img_path)
    if "/Hidden" in folder:
        return folder.replace("/images", "/osilab_seg") + name + "_label.tiff"
    elif "/Public/images" in folder:
        return folder.replace("/images", "/1st_osilab_seg") + name + "_label.tiff"
    elif "/Public/WSI" in folder:
        return folder.replace("/WSI", "/osilab_seg_WSI") + name + "_label.tiff"
    return None

for cat in [UNLABELED, LABELED]:
    for ext in img_types:
        for path in file_types[cat][ext]:
            synth_path = get_synth_path(path)
            if synth_path and os.path.exists(datapath_raw + synth_path):
                synth_assoc[path] = synth_path
                visited.remove(synth_path)

print(len(synth_assoc))
print(visited)
assert not visited

In [ ]:
import pandas as pd
import PIL.Image as Image

def save_dataframe(assoc, framename):
    expected = len(assoc)

    numbers = []
    for img_path, datapath in assoc.items():
        print(len(numbers), "/", expected)
        # Get global statistics
        imgT = tif.imread(datapath_raw + datapath)
        numbers.append({"Path":img_path, "Mask":datapath, "Width": imgT.shape[1], "Height":imgT.shape[0], "Objects": imgT.max(), "Background": (imgT == 0).sum()})

    data_map = pd.DataFrame(numbers, columns = ["Path", "Mask", "Width", "Height", "Objects", "Background"]).set_index("Path")
    data_map.to_csv(datapath_raw + framename + ".csv")

    return data_map

data_map = save_dataframe(mask_assoc, ".labels")
synth_map = save_dataframe(synth_assoc, ".synth")



In [ ]:

index = 0
for img_path, datapath in mask_assoc.items():
    print(index, "/", len(mask_assoc))
    # Save black-white mask
    imgT = tif.imread(datapath_raw + datapath)
    im = Image.fromarray((imgT != 0).astype('uint8')*255)
    folder, name, ext = split_filepath(datapath)
    target = datapath_raw + ".labels/" + folder
    os.makedirs(target, exist_ok=True)
    im.save(target + name + ".png")
    index += 1


In [ ]:
#data_map["Objects"] < 2000
data_map

In [ ]:
print("Valeurs aberrantes?")
data_map[data_map["Objects"] > 2000]

In [ ]:

num_obj = data_map["Objects"]
print(num_obj[num_obj > 2000])
num_obj = num_obj[num_obj < 2000]
print(len(num_obj), sum(num_obj), set(num_obj))

# Display parallel histograms
plt.figure()
plt.subplot(2, 1, 1)
plt.hist(np.log2([x for x in num_obj]), bins=100)
plt.title("Number of segmented objects per file")
plt.xlabel("Number of objects (log2)")
#plt.xticks([2.0 ** x for x in range(5)])
plt.ylabel("Count")
plt.show()

In [ ]:
print(len(set(n for n in (data_map["Width"] * data_map["Height"]))))

area = data_map["Width"] * data_map["Height"]
print("Too big:", area[area > 10 ** 7].count())
#area = area[area < 10 ** 7]

# Display parallel histograms
plt.figure()
plt.subplot(2, 1, 1)
plt.hist(np.log2(area), bins=100)
#plt.hist([np.log2(n) for n in numbers if n < 2000], bins=100)
plt.title("Size of of mask files")
plt.xlabel("Size of file (log2)")
#plt.xticks([2.0 ** x for x in range(5)])
plt.ylabel("Count")
plt.show()

In [ ]:
#print(set(n[2] for n in numbers))

density = 1 - (data_map["Background"] / data_map["Width"] / data_map["Height"])

plt.figure()
plt.subplot(2, 1, 1)
plt.hist(density, bins=100)
#plt.hist([-np.log2(n[2]) for n in numbers], bins=100)
plt.title("Density of segmented objects per file")
plt.xlabel("Density of objects")
#plt.xticks([2.0 ** x for x in range(5)])
plt.ylabel("Count")
plt.show()

In [ ]:
radius = (density / data_map["Objects"] / np.pi) ** .5

plt.figure()
plt.subplot(2, 1, 1)
plt.hist(radius, bins=100)
#plt.hist([-np.log2(n[2]) for n in numbers], bins=100)
plt.title("Average radius of segmented objects per file")
plt.xlabel("Radius of objects (in %)")
#plt.xticks([2.0 ** x for x in range(8)])
plt.ylabel("Count")
plt.show()